<div style="text-align: center"> <H2> Домашнее задание №8, к лекции "Понятие класса" </H2></div>
<div style='text-align: center'> <H4> по курсу PYDA-15 - Python для анализа данных </H4></div>
</div>
<p>
<div style="text-align: right"> Выполнил: студент группы PAL-4 Константин Лемешок </div>

---

### Задание 1

<div class="alert alert-block alert-success">

Напишите функцию, которая возвращает название валюты (поле ‘Name’) с максимальным значением курса с помощью сервиса https://www.cbr-xml-daily.ru/daily_json.js
    
</div>

Решение см. в коде вместе с решением к Заданию 2. Название функции - most_expensive_valute().

---

### Задание 2

<div class="alert alert-block alert-success">

Добавьте в класс Rate параметр diff (со значениями True или False), который в случае значения True в методах курсов валют (eur, usd итд) будет возвращать не курс валюты, а изменение по сравнению в прошлым значением. Считайте, self.diff будет принимать значение True только при возврате значения курса. При отображении всей информации о валюте он не используется.
    
</div>

---

In [ ]:
import requests
class Rate:
    """
    Позволяет получать информацию о курсах валют за текущую дату.
    Атрибуты класса:
    - format_ : Значения:
        * 'value' - возвращает только текущее значение курса валюты
        * 'full' - возвращает полную информацию о валюте
    - diff : Значения:
        * False - не выводит разницу между текущим и предыдущим значением валюты
        * True - выводит разницу между текущим и предыдущим значением валюты
    Методы класса:
    - exchange_rates() : возвращает ответ сервиса с информацией о валютах (детальней см. Docstring метода)
    - make_format() : возвращает информацию о валюте currency в трех вариантах (детальней см. Docstring метода)
    - most_expensive_valute() : возвращает самую дорогостоящую валюту в расчете за единицу
    - eur(), usd() и аналогичные по другим валютам : Возвращает курс заданной валюты на сегодня в формате self.format    
    """
    
    def __init__(self, format_='value', diff=False):
        self.format = format_
        self.diff = diff
    
    def exchange_rates(self):
        """
        Возвращает ответ сервиса с информацией о валютах в виде:
        {
            'AMD': {
                'CharCode': 'AMD',
                'ID': 'R01060',
                'Name': 'Армянских драмов',
                'Nominal': 100,
                'NumCode': '051',
                'Previous': 14.103,
                'Value': 14.0879
                },
            ...
        }
        """
        self.r = requests.get('https://www.cbr-xml-daily.ru/daily_json.js').json()
        return self.r['Valute']
    
    def make_format(self, currency):
        """
        Возвращает информацию о валюте currency в трех вариантах:
        - полная информация о валюте при self.format='full':
        Rate('full').make_format('EUR')
        {
            'CharCode': 'EUR',
            'ID': 'R01239',
            'Name': 'Евро',
            'Nominal': 1,
            'NumCode': '978',
            'Previous': 79.6765,
            'Value': 79.4966
        }
        - только курс валюты при при self.format='value' и diff=False
        Rate('value').make_format('EUR')
        79.4966
        - курс валюты и разница между текущим и предыдущим значением курса при при self.format='value' и diff=True
        Rate('value',True).make_format('EUR')
        """
        response = self.exchange_rates()
        
        if currency in response:
            if self.format == 'full':
                return response[currency]
            
            if self.format == 'value' and self.diff == False:
                return response[currency]['Value']
            
            if self.format == 'value' and self.diff == True:
                return response[currency]['Value'], response[currency]['Value'] - response[currency]['Previous']
        
        return 'Error'
    
    def most_expensive_valute(self):
        """Возвращает самую дорогостоящую валюту в расчете за единицу"""
        valute_value = 0
        response = self.exchange_rates()
        for name, data in response.items():
            if data['Value'] / data['Nominal'] > valute_value:
                valute_value = data['Value'] / data['Nominal']
                valute_name = data['Name']
        return valute_name, valute_value
    
    def eur(self):
        """Возвращает курс евро на сегодня в формате self.format"""
        return self.make_format('EUR')
    
    def usd(self):
        """Возвращает курс доллара на сегодня в формате self.format"""
        return self.make_format('USD')
    
    def brl(self):
        """Возвращает курс бразильского реала на сегодня в формате self.format"""
        return self.make_format('BRL')       

In [ ]:
a = Rate(diff=True)

In [ ]:
a.most_expensive_valute()

In [ ]:
Rate('value',True).make_format('EUR')

In [ ]:
a.eur()

---

### Задание 3

<div class="alert alert-block alert-success">
Напишите класс Designer, который учитывает количество международных премий для дизайнеров. Повышение на 1 грейд за каждые 7 баллов. Получение международной премии – это +2 балла.
    
    Чтобы задача была ближе к реальности, изменил изначальное условие: "Считайте, что при выходе на работу сотрудник уже имеет две премии, и их количество не меняется со стажем" на следующее: "При выходе на работу у сотрудника нет международных премий, но он может получить их в процессе работы".

</div>

---

In [ ]:
class Employee:
    """
    С помощью класса можно выполнять базовые действия в рамках системы повышения сотрудников.
    Атрибуты класса:
    - name : Имя сотрудника
    - seniority : Количество баллов сотрудника в результате пройденных аккредитаций и других достижений
    Методы класса:
    - grade_up() : Повышает уровень сотрудника
    - publish_grade() : Публикует уровень сотрудника
    """
    def __init__(self, name, seniority):
        self.name = name
        self.seniority = seniority        
        self.grade = 1    
    
    def grade_up(self):
        """Повышает уровень сотрудника"""
        self.grade += 1
    
    def publish_grade(self):
        """Публикует результаты аккредитации сотрудников"""
        print(f'Имя: {self.name}, баллы: {self.seniority}, уровень: {self.grade}')

In [ ]:
class Designer(Employee):
    """
    Наследует класс Employee() и реализует систему повышения для дизайнера.
    Методы класса:
    - check_if_it_is_time_for_upgrade() : Проверяет, не пора ли повысить уровень дизайнера, если пора - повышает
    - international_prize() : После получения международной премии увеличивает количество баллов дизайнера на 2,
        возвращает текущий уровень дизайнера
    - accreditation() : Вследствие прохождения аккредитации увеличивает количество баллов дизайнера на 1,
        возвращает текущий уровень дизайнера
    """
    def __init__(self, name, seniority):
        super().__init__(name, seniority)
        
    def check_if_it_is_time_for_upgrade(self):                
        """
        Проверяет, не пора ли повысить уровень дизайнера. Повышает его в случае, если:
        - дизайнер получил количество баллов кратное 7 после очередной аккредитации
        - дизайнер получил количество балов, делящееся на 7 с остатком 1 после получения международной премии
        Возвращает текущий уровень дизайнера.
        """
        if (self.seniority !=0 and self.seniority % 7 == 0) or (self.seniority % 7 == 1 and self.jump == True):
            self.grade_up()        
        return self.publish_grade()

    def international_prize(self):
        """
        Увеличивает количество баллов дизайнера на 2 после получения международной премии.
        Возвращает текущее количество баллов и уровень дизайнера.
        """        
        self.seniority += 2
        self.jump = True
        self.check_if_it_is_time_for_upgrade()
        
    def accreditation(self):
        """
        Увеличивает количество баллов дизайнера на 1 после прохождения аккредитации.
        Возвращает текущее количество баллов и уровень дизайнера.
        """        
        self.seniority += 1
        self.jump = False
        self.check_if_it_is_time_for_upgrade()

In [ ]:
John = Designer('John', 0)

In [ ]:
John.accreditation()

In [ ]:
John.international_prize()

---